# CONFIGURACIONES

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime
from unidecode import unidecode
from fuzzywuzzy import fuzz, process
import numpy as np
import re
import sqlalchemy.exc as sa_exc
from sqlalchemy.exc import SQLAlchemyError
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import unicodedata

## VISUALIZACIÓN

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# EXTRACT

In [54]:
df = pd.read_excel(r"C:\Users\aleja\OneDrive\Documents\Oracle - Program\orange\01 OVERALL FINAL DATA PHD.xlsx",
                   sheet_name='RAW DATA', )

# TRANSFORM

## SEPARAR LAS EMPRESAS DE LAS MEDICIONES

In [30]:
companies = df.iloc[7:, :28] 
companies.columns = companies.iloc[0]
companies = companies[2:]

In [67]:
scores = df.iloc[:, 27:]
scores = scores.iloc[1:]

## ORGANIZAR EL DF

In [68]:
# Paso 1: Identificar las columnas donde alguna fila contiene 'Addressed criteria'
columnas_a_mover = scores.columns[scores.apply(lambda col: col.astype(str).str.contains('Addressed criteria', na=False)).any()]

# Paso 2: Crear un nuevo DataFrame con estas columnas
addressed_criteria = scores[columnas_a_mover].copy()

# Paso 3: Eliminar estas columnas del DataFrame original
scores.drop(columns=columnas_a_mover, inplace=True)

In [69]:
# Paso 1: Identificar las columnas donde alguna fila contiene 'Use of tools'
columnas_a_mover = scores.columns[scores.apply(lambda col: col.astype(str).str.contains('Use of tools', na=False)).any()]

# Paso 2: Crear un nuevo DataFrame con estas columnas
use_tools = scores[columnas_a_mover].copy()

# Paso 3: Eliminar estas columnas del DataFrame original
scores.drop(columns=columnas_a_mover, inplace=True)

### TRANSPONER EL DF

In [70]:
# Eliminar las filas 1, 4 y 7
scores = scores.reset_index(drop=True).drop(index=[1, 4, 7]).reset_index(drop=True)

In [72]:
# Paso 1: Transponer el DataFrame (convertir filas en columnas)
dfT = scores.T.reset_index(drop=True)

# Paso 2: Renombrar las columnas para que representen los niveles de jerarquía
dfT.columns = ["Dynamic_Capability_Stage", "Business_Parameters", "Business_Sub", "Metrics","Question"] + [f"Valor_{i}" for i in range(len(dfT.columns)-5)]

# Paso 3: Convertir a formato largo (melt)
df_long = dfT.melt(id_vars=["Dynamic_Capability_Stage", "Business_Parameters", "Business_Sub", "Metrics","Question"], 
                   var_name="Medicion", value_name="Valor")

# Paso 4: Eliminar la columna de "Medicion" ya que no aporta información
df_long = df_long.drop(columns=["Medicion"])

In [74]:
df_long

,Dynamic_Capability_Stage,Business_Parameters,Business_Sub,Metrics,Question,Valor
0,Sensing,Performance,Business development,Frequency (business),"Indicate how often your business identifies and defines performance challenges or opportunities, particularly in relation to customer acquisition.",4
1,Sensing,Performance,Business development,Effectiveness,"Indicate how effective your business is to identify and define challenges or performance opportunities, particularly in relation to customer acquisition.",4
2,Sensing,Performance,Business development,Frequency (individual),"Indicate how often you participate in the identification and definition of challenges or opportunities for performance within your business development processes, particularly in relation to the acquisition of customers.",4
3,Sensing,Performance,Business development,Certainty,"Indicate the typical level of certainty that you experience by identifying and defining challenges or opportunities for business development, particularly in relation to the acquisition of clients.",3
4,Sensing,Performance,Business development,Frequency (business),Indicate how often your business establishes clear objectives and performance priorities for the acquisition of customers in business development.,5
...,...,...,...,...,...,...
8059,Transformation,People,Management,Certainty,"Please indicate the typical level of certainty you experience when developing optimal management solutions, reconfigurations, and action plans for customer acquisition.",3
8060,Transformation,People,Management,Frequency (business),Indicate how often your company evaluates and monitors management's performance and practices against customer acquisition objectives.,3
8061,Transformation,People,Management,Effectiveness,Indicate how effective your business is in evaluating and monitoring management performance and practices against customer acquisition objectives.,2
8062,Transformation,People,Management,Frequency (individual),Indicate how frequently you participate in evaluating and monitoring management practices and performance against customer acquisition objectives.,3
